# Лабораторная работа № 2 "Онтологии"

В блокноте показан пример выполнения некоторых заданий второй лабораторной работы. Предполагается, что перед запуском этого блокнота с помощью редактора Protege была создана и сохранена в файл `sample_ontology` онтология. В онтологии определены классы `Person`, `Man`, `Woman`, `Parent`, объектные свойства `hasParent` и `hasChild`, а также два индивида (`Abraham` и `Jacob`).

В Python вся работа с онтологиями производится с помощью библиотеки `owlready2`:

In [1]:
import owlready2

## Загрузка онтологии

In [2]:
onto = owlready2.get_ontology("sample_ontology").load()

Выведем содержимое онтологии, чтобы убедиться, что всё загрузилось правильно:

In [3]:
print('Classes:', list(onto.classes()))

Classes: [sample_ontology.Person, sample_ontology.Female, sample_ontology.Male]


In [4]:
print('Individuals:', list(onto.individuals()))

Individuals: [sample_ontology.Abraham, sample_ontology.Jacob]


## Классификация индивидов

Библиотека Owlready2 позволяет осуществлять исследование графа, задаваемого аксиомами онтологии (например, `ClassAssertion`).

In [5]:
abraham = onto.search_one(iri = "*Abraham")
jacob = onto.search_one(iri = "*Jacob")

К каким классам относятся эти индивиды?

In [6]:
abraham.is_a

[owl.Thing]

In [7]:
jacob.is_a

[owl.Thing]

Действительно, онтология не содержит аксиом, которые бы напрямую связывали индивидов `Abraham` и `Jacob` с какими-либо классами, поэтому всё, что о них можно сказать, это то, что они принадлежат к базовому классу OWL - `owl.Thing` (что-то).

### Свойства индивида

Изучим также свойства этих индивидов. Напомним, что в онтологии есть утверждение о том, что `Abraham` `hasChild` `Jacob`, однако нет явной информации о том, что `Jacob` `hasParent` `Abraham`.

In [8]:
for prop in abraham.get_properties():
    for value in prop[abraham]:
        print(".%s == %s" % (prop.python_name, value))

.hasChild == sample_ontology.Jacob


In [9]:
for prop in jacob.get_properties():
    for value in prop[jacob]:
        print(".%s == %s" % (prop.python_name, value))

.hasParent == sample_ontology.Abraham


Довольно неожиданный результат - видимо, при построении графа библиотека `Owlready2` сама обработала информацию о взаимной обратности этих свойств.

# Включение reasoner (машины вывода)

Вообще, машина вывода является внешней программой, написанной на Java и для ее запуска, соответственно, нужна Java-машина. Подробно о настройке можно почитать в документации по `Owlready2` (https://owlready2.readthedocs.io/en/v0.42/reasoning.html#configuration). Есть, однако, шанс встретиться в проблемой, заключающейся в невозможности выделить необходимое количество памяти при запуске машины вывода (по умолчанию требуется 2Гб). Решить эту проблему можно уменьшив значение переменной `JAVA_MEMORY` в файле `site-packages\owlready2\reasoning.py` до приемлемого. 

In [10]:
owlready2.sync_reasoner()

* Owlready2 * Running HermiT...
    java -Xmx1000M -cp E:\opt\Miniconda3\envs\aim\lib\site-packages\owlready2\hermit;E:\opt\Miniconda3\envs\aim\lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/hatter/AppData/Local/Temp/tmpxxukulay
* Owlready2 * HermiT took 0.6947436332702637 seconds
* Owlready * Reparenting sample_ontology.Abraham: {owl.Thing} => {sample_ontology.Person}
* Owlready * Reparenting sample_ontology.Jacob: {owl.Thing} => {sample_ontology.Person}
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [11]:
abraham.is_a

[sample_ontology.Person]

In [12]:
jacob.is_a

[sample_ontology.Person]

Как и ожидалось, после включения машины вывода произошла классификация индивидов.